In [1]:
# from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import pandas as pd

hunt = pd.read_csv('Data\\Hunt\\huntfile.csv')
khar = pd.read_csv('Data\\Kharchenko\\kharchenkofile.csv')
dias = pd.read_csv('Data\\Dias\\diasfile.csv')
xmatch = pd.read_csv('Data\\Hunt\\xmatchfile.csv')

"""In case you want to use fits files"""
# def datahandler(filename):
#     authorname = filename.split('\\')
#     hdul = fits.open(filename)
#     hdr1 = hdul[0].header #first and second header
#     hdr2 = hdul[1].header
#     data, cols = hdul[1].data, hdul[1].columns
#     metadata = [ [ hdr2[f"TTYPE{i+1}"], hdr2.comments[f"TTYPE{i+1}"] ] for i, j in enumerate(cols.names) ]
#     # mydict[authorname]['hdul'] = hdul
#     return hdul[1].data, hdul[1].columns, metadata

xmatch

,ID,NameLit,SourceCat,TypeSourceCat,Sep,SepTidal,SepTidalLit,SepTidalData,pmRASep,pmRASigma,pmDESep,pmDESigma,PlxSep,PlxSigma,maxSigma,meanSigma
0,0,1636-283,Kharchenko+13,hipparcos,0.000298,0.002366,0.003970,0.002366,3.4450,0.662721,1.69600,0.3213,0.04456,0.0,0.663,0.3280
1,0,1636-283,Kharchenko+13,position,0.000298,0.002366,0.003970,0.002366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,2MASS-GC01,Bica+18,position,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,2MASS-GC02,Bica+18,position,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,2MASS-GC02,Kharchenko+13,"position,hipparcos",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29940,7162,vdBergh_85,Kharchenko+13,position,0.009891,0.024375,0.076088,0.024375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29941,7162,vdBergh_85,Dias+02,position,0.010300,0.025379,0.308965,0.025379,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29942,7163,vdBergh_92,Kharchenko+13,position,0.089140,0.185433,0.356566,0.185433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29943,7163,vdBergh_92,Cantat-Gaudin+20,gaia_dr2,0.032700,0.068013,0.145961,0.068013,0.1545,0.527101,0.03532,0.0000,0.03385,0.0,0.527,0.1757


### Getting Kharchenko numbers
Kharchenko clusters: $n_\mathrm{clusters}=2935 $, $n_\mathrm{detected}= 1513 \newline$ 
Steps:
1. Get Kharchenko from crossmatch table
2. Remove records with Sep (seperation between cluster centres) == NaN
3. Check if crossmatched kharchenko 

In [2]:
kharchenko_xm = xmatch.query('SourceCat == "Kharchenko+13"').dropna(subset='Sep') #Get crossmatch table with seperation=NaN removed
kharchenko_names_oc = np.array(khar.query('Type != "g"')['Name'], dtype=object) #get clusters names from kharchenko data with type is not 'g'=globular

IDs_hunt = np.unique(hunt.ID.values)
IDs_xm = np.unique(kharchenko_xm.ID.values)

In [3]:
testlist = []
c=0
for i in IDs_xm:
    if i > 7166: continue
    type_object = hunt.query('ID == @i')['Type'].values
    if type_object == 'o':
        for j in np.unique(kharchenko_xm.query('ID == @i')['NameLit'])[0:1]:
            testlist.append(j)

print(testlist)

['1636-283', 'FSR_1315', 'ASCC_5', 'ASCC_6', 'ASCC_9', 'ASCC_11', 'ASCC_12', 'ASCC_13', 'ASCC_14', 'ASCC_19', 'ASCC_20', 'ASCC_23', 'ASCC_27', 'ASCC_28', 'ASCC_29', 'ASCC_30', 'ASCC_36', 'ASCC_51', 'ASCC_58', 'ASCC_59', 'ASCC_60', 'ASCC_62', 'ASCC_63', 'ASCC_65', 'ASCC_66', 'ASCC_67', 'ASCC_69', 'ASCC_71', 'ASCC_73', 'ASCC_75', 'ASCC_79', 'ASCC_84', 'ASCC_85', 'ASCC_87', 'ASCC_88', 'ASCC_90', 'ASCC_93', 'ASCC_94', 'ASCC_95', 'ASCC_100', 'ASCC_101', 'ASCC_104', 'ASCC_105', 'ASCC_106', 'ASCC_107', 'ASCC_108', 'ASCC_110', 'ASCC_113', 'ASCC_114', 'ASCC_115', 'ASCC_127', 'ASCC_128', 'Alessi_2', 'Alessi_3', 'Alessi_5', 'Alessi_6', 'Alessi_9', 'Alessi_10', 'Alessi_12', 'Alessi_17', 'Alessi_19', 'Alessi_20', 'Alessi_21', 'ASCC_92', 'ASCC_32', 'ASCC_124', 'ASCC_97', 'ASCC_50', 'Alessi_53', 'Alessi_60', 'Alessi_62', 'ASCC_129', 'ASCC_42', 'ASCC_118', 'ASCC_68', 'ASCC_10', 'ASCC_112', 'Andrews-Lindsay_5', 'Antalova_1', 'Antalova_2', 'Archinal_1', 'Auner_1', 'Aveni-Hunter_1', 'BDSB_30', 'BDSB_82',

In [4]:
cross = np.intersect1d(IDs_xm, IDs_hunt, return_indices=True)

print(cross)

poep = kharchenko_xm.iloc[cross[1]]
poep

(array([   0,    2,    3, ..., 7164, 7165, 7166], dtype=int64), array([   0,    1,    2, ..., 1662, 1663, 1664], dtype=int64), array([   0,    2,    3, ..., 7164, 7165, 7166], dtype=int64))


,ID,NameLit,SourceCat,TypeSourceCat,Sep,SepTidal,SepTidalLit,SepTidalData,pmRASep,pmRASigma,pmDESep,pmDESigma,PlxSep,PlxSigma,maxSigma,meanSigma
0,0,1636-283,Kharchenko+13,hipparcos,0.000298,0.002366,0.003970,0.002366,3.4450,0.662721,1.6960,0.32130,0.04456,0.0000,0.663,0.3280
1,0,1636-283,Kharchenko+13,position,0.000298,0.002366,0.003970,0.002366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,114,ASCC_10,Kharchenko+13,hipparcos,0.164400,0.031312,0.310144,0.031312,1.0470,0.274158,0.2413,0.05259,0.11460,0.4058,0.406,0.2442
28,114,ASCC_10,Kharchenko+13,position,0.164400,0.031312,0.310144,0.031312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,50,ASCC_100,Kharchenko+13,position,0.025990,0.008916,0.071212,0.008916,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13179,6759,NGC_2448,Kharchenko+13,position,0.077910,0.432835,0.432835,0.496529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13182,4872,NGC_2451A,Kharchenko+13,position,0.571200,0.212439,0.439395,0.212439,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13184,7104,NGC_2451A,Kharchenko+13,position,0.771200,0.499494,0.593268,0.499494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13185,4872,NGC_2451A,Kharchenko+13,hipparcos,0.571200,0.212439,0.439395,0.212439,0.8128,0.171078,0.1972,0.03302,0.05829,0.0140,0.171,0.0727


## Comparing catalogues

The goal now is to get the correct clusters from the Kharchenko, Dias and CanatGaudin catalogues with respect to the Hunt paper. Pretty much recreate the table 4 from the hunt paper. We want to cross match the catalogues. We first want to find the $n_\mathrm{clusters}$ for each catalogue.

In [5]:
poop = [48, 84, 26, 50, 27, 59]
poop[0,3,2]


TypeError: list indices must be integers or slices, not tuple

In [ ]:
#getting ocs from kharchenko
mask_khar = (data_khar['Type']!='a') & (data_khar['Type'] != 'm') & (data_khar['Type'] != 's')
khar_ocs = data_khar[mask_khar]

print(f"The amount of clusters in the Kharchenko catalogue is indeed {len(khar_ocs)}")

#significant cluster from hunt 
# mask_hunt = data_hunt['CST'] >5
# print(f"Amount of significant cluster is {len(data_hunt[mask_hunt])}")

The amount of clusters in the Kharchenko catalogue is indeed 2935


In [ ]:
all_names = np.concatenate([name.split(',') for name in data_hunt['AllNames']]) #get all names from hunt file

cross = np.in1d(list(khar_ocs['Name']), all_names) #intersect hunt all names and kharchenko files
np.unique(khar_ocs[cross]['Name'])

print(f"The amount to found Kharchenko OCs are {len(np.unique(khar_ocs[cross]['Name']))} != 1513 as in the paper")

The amount to found Kharchenko OCs are 1538 != 1513 as in the paper
